In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# Carregar base de CMED
base_cmed = pd.read_excel("Lista de Preço CMED.xls", 
                          skiprows=48,
                          decimal=',')

eans = base_cmed['EAN 1'][:30]
#preco = base_cmed[['PF 0%','PF 22%']]

In [3]:
def extra_farma_preco(ean):
    url = 'https://www.extrafarma.com.br/'
    ean = ean

    url_final = f'{url}{ean}?_q={ean}&map=ft'
    
    response = requests.get(url_final)
    soup = BeautifulSoup(response.content, 'html.parser')
        
    nome_elemento = soup.find('span', {'class': 'vtex-product-summary-2-x-productBrand'})
    preco_elemento = soup.find('div', {'class': 'extrafarmav2-teaser-labels-2-x-price'})
    
    if nome_elemento is None:
        nome = 'Produto não encontrado'
    else:
        nome = nome_elemento.text.strip()
    
    if preco_elemento is None:
        preco = 0
    else:
        preco = preco_elemento.text.strip().removeprefix('R$\xa0')
        preco = float(preco.replace(',', '.'))
    
    data = {
        'NOME_EXTRAFARMA': [nome],
        'PRECO_EXTRAFARMA': [preco],
        'EAN1':[ean]
            }

    df = pd.DataFrame(data)
    return(df)

In [4]:
precos = pd.DataFrame()

for i in eans:
    preco = extra_farma_preco(i)
    precos = pd.concat([precos, preco], axis=0)

In [5]:
#Carregar base de dados CMED com campos que serão analizados
base_cmed_filtrada = base_cmed[['SUBSTÂNCIA',
                                'CNPJ',
                                'LABORATÓRIO',
                                'REGISTRO',
                                'EAN 1',
                                'PF Sem Impostos', 
                                'PF 0%', 
                                'PF 22%', 
                                'PMC 0%',
                                'PMC 22%'
                               ]]

In [6]:
# Agrupar tabelas por EAN1
base_comparacao = pd.merge(precos, base_cmed_filtrada, 
                           left_on='EAN1', 
                           right_on='EAN 1', 
                           how='left')

In [8]:
base_comparacao[['EAN1',
                 'SUBSTÂNCIA',
                 'PRECO_EXTRAFARMA',
                 'PF Sem Impostos',
                 'PF 22%',
                 'PMC 0%',
                 'PMC 22%']]

,EAN1,SUBSTÂNCIA,PRECO_EXTRAFARMA,PF Sem Impostos,PF 22%,PMC 0%,PMC 22%
0,7896112401247,VALPROATO DE SÓDIO,51.30,38.10,48.85,52.67,67.53
1,7896210500354,SALICILATO DE FENILA;ÁCIDO SALICÍLICO;ÓXIDO DE...,0.00,5.04,7.50,7.58,10.12
2,7896116871497,DIENOGESTE;VALERATO DE ESTRADIOL,0.00,107.37,159.69,161.32,215.53
3,7897337706575,MONTELUCASTE,63.65,52.30,67.05,72.30,92.69
4,7897337706568,MONTELUCASTE,65.35,52.30,67.05,72.30,92.69
5,7897337706490,MONTELUCASTE,0.00,35.16,45.08,48.61,62.32
6,7897337706506,MONTELUCASTE,65.55,52.30,67.05,72.30,92.69
7,7897337706520,MONTELUCASTE,0.00,35.16,45.08,48.61,62.32
8,7897337706551,MONTELUCASTE,0.00,35.16,45.08,48.61,62.32
9,7897337707923,MONTELUCASTE,0.00,164.67,211.12,227.65,291.86
